Title: Predicting tennis player rank categories based on the number of seasons active, prize money won, age, and height.

Background information:
Tennis rankings are determined based on the number of points a player has. Points are acquired by playing in tournaments. Winning higher-status tournaments, such as a Grand Slam, results in more points gained (2000). Winning tournaments also lets players earn prize money, where like points, winning a higher-status tournament gives more prize money. Higher-ranking players are therefore more likely to have earned more prize money. Additionally, players who have competed in more seasons have had more opportunities to win tournaments, possibly indicating that they have a higher ranking



Question: 
The question we will be exploring in this project is: Can we use known player statistics (number of seasons active, prize money, age, and height) of the top 500 players to predict which category rank a future player will reside i? 

To answer this question we will be using the “Player Stats for the Top 500 Players” data frame whi ch provides statistics on each top player such as age, height, number of seasons active, and current ra


Variables:

We will be using the number of seasons a player has played in, how much prize money they've won, the age of the player, and the height of the player.


ayer
nk.
.


